In [2]:
!pip install graphviz --user
!pip install pyvis --user
!pip install ipython --user

In [2]:
import pandas as pd
import numpy as np
from graphviz import Digraph #установить graphviz с сайта pypi.org, либо выполнить pip install с правами администратора
import networkx as nx #установить networkx
import warnings
warnings.filterwarnings("ignore")
from pyvis.network import Network #установить pyvis 
from IPython.core.display import display, HTML
from IPython.display import Javascript as JS

In [3]:
transaction_data = pd.read_csv('prepared_data.csv', delimiter= ';') 
#синтетический DF (в оригинале если грузить из ЦФТ получаем выгрузку транзакций содержащие дату, 
#номер счета отправителя, получателя, сумму и дату, 
#далее отдельно выгружаем справочник инн + наименование оргнаизации их складываем в поле Payer, Recipient, реализуется в SQL запросе)

In [4]:
df_f = pd.read_csv('prepared_data.csv', delimiter= ';') 

In [5]:
clients = pd.concat([transaction_data['Payer'], transaction_data['Recipient']]).unique()

# Разбиваем клиентов на две группы соотношением 30/70
group1_size = int(0.3 * len(clients))
group1_clients = np.random.choice(clients, size=group1_size, replace=False)
group2_clients = np.setdiff1d(clients, group1_clients)

# Создаем столбцы "flag1" и "flag2" и заполняем их нулями
transaction_data['flag1'] = 0
transaction_data['flag2'] = 0

# Заполняем столбец "flag1" единицами для клиентов из первой группы
transaction_data.loc[(transaction_data['Payer'].isin(group1_clients)) | (transaction_data['Recipient'].isin(group1_clients)), 'flag1'] = 1

# Заполняем столбец "flag2" единицами для клиентов из второй группы
transaction_data.loc[(transaction_data['Payer'].isin(group2_clients)) | (transaction_data['Recipient'].isin(group2_clients)), 'flag2'] = 1

In [6]:
transaction_data['payer-buyer'] = transaction_data['Payer']+'-->'+ transaction_data['Recipient']

In [7]:
vacalabry = transaction_data[['payer-buyer', 'flag1', 'flag2']]
vacalabry = vacalabry.drop_duplicates()

In [8]:
vacalabry # справочник по легенде 0 - не является заемщиком, 1 - является заемщиком

,payer-buyer,flag1,flag2
0,Client_171-->Client_73,1,1
1,Client_74-->Client_171,1,1
2,Client_180-->Client_171,1,0
3,Client_73-->Client_171,1,1
4,Client_16-->Client_171,1,1
...,...,...,...
694,Client_72-->Client_6,0,1
698,Client_72-->Client_32,0,1
702,Client_81-->Client_72,0,1
707,Client_167-->Client_72,1,1


In [9]:
#Группируем DF считаем суммы и количество связей
df_t_1 = pd.DataFrame(transaction_data['payer-buyer'].value_counts()).reset_index()
df_t_1.columns = ['Transact', 'Frequency',]
df_t_1['Sum_tr'] = df_t_1['Transact'].map(transaction_data.groupby('payer-buyer')['Sum'].agg('sum'))
transact = df_t_1['Transact'].values.tolist()
counts = df_t_1['Frequency'].values.tolist()
summ_tr = df_t_1['Sum_tr'].values.tolist()

In [10]:
#формируем финальный DF
df = df_t_1['Transact'].str.split('-->',expand=True)
df.columns=['Payer','Recipient']
for column in df.columns:
    df[column] = df[column].str.replace(column+'=','')

final_df = pd.concat([df_t_1,df],axis=1)

In [11]:
final_df

,Transact,Frequency,Sum_tr,Payer,Recipient
0,Client_161-->Client_86,62,450477.68993,Client_161,Client_86
1,Client_22-->Client_22,55,136094.00000,Client_22,Client_22
2,Client_22-->Client_20,40,51100.00000,Client_22,Client_20
3,Client_153-->Client_98,37,22900.00000,Client_153,Client_98
4,Client_22-->Client_4,29,42782.00000,Client_22,Client_4
...,...,...,...,...,...
208,Client_22-->Client_135,1,10000.00000,Client_22,Client_135
209,Client_22-->Client_2,1,19000.00000,Client_22,Client_2
210,Client_22-->Client_191,1,1000.00000,Client_22,Client_191
211,Client_22-->Client_79,1,1000.00000,Client_22,Client_79


In [12]:
final_df_for_graph = final_df.merge(vacalabry, left_on='Transact', right_on='payer-buyer', how='left')
final_df_for_graph = final_df_for_graph.drop(['payer-buyer'],axis=1)

In [13]:
final_df_for_graph

,Transact,Frequency,Sum_tr,Payer,Recipient,flag1,flag2
0,Client_161-->Client_86,62,450477.68993,Client_161,Client_86,0,1
1,Client_22-->Client_22,55,136094.00000,Client_22,Client_22,0,1
2,Client_22-->Client_20,40,51100.00000,Client_22,Client_20,0,1
3,Client_153-->Client_98,37,22900.00000,Client_153,Client_98,1,0
4,Client_22-->Client_4,29,42782.00000,Client_22,Client_4,0,1
...,...,...,...,...,...,...,...
208,Client_22-->Client_135,1,10000.00000,Client_22,Client_135,1,1
209,Client_22-->Client_2,1,19000.00000,Client_22,Client_2,0,1
210,Client_22-->Client_191,1,1000.00000,Client_22,Client_191,0,1
211,Client_22-->Client_79,1,1000.00000,Client_22,Client_79,0,1


In [50]:
got_net = Network(notebook=True, height="900px", width="1800px", bgcolor="#11111", font_color="black", directed=True)
got_net.barnes_hut(gravity=-1000, central_gravity=0, spring_length=250, spring_strength=0.01, damping=0.09, overlap=0)
got_net.set_edge_smooth('continuous')

got_net.set_options("""
    var options = {
        "nodes": {
            "font": {
                "size": 10
            }
        },
        "edges": {
            "font": {
                "size": 10
            }
        }
    }
    """
)

got_data = final_df_for_graph

sources = got_data['Payer']
targets = got_data['Recipient']
weights = got_data['Sum_tr']
F = got_data['flag1']
N = got_data['flag2']
edge_data = zip(sources, targets, weights, N, F)

for src, dst, w, F, N in edge_data:
    # Определение цвета узлов в зависимости от значений F и N
    if F == 1 and N == 0:
        got_net.add_node(src, src, title=src, color="blue")  # узлы отправитель
        got_net.add_node(dst, dst, title=dst, color="red")  # узлы получатель
    elif F == 0 and N == 1:
        got_net.add_node(src, src, title=src, color="red")  # узлы отправитель
        got_net.add_node(dst, dst, title=dst, color="blue")  # узлы получатель
    elif F == 0 and N == 0:
        got_net.add_node(src, src, title=src, color="blue")  # узлы отправитель
        got_net.add_node(dst, dst, title=dst, color="blue")  # узлы получатель
    elif F == 1 and N == 1:
        got_net.add_node(src, src, title=src, color="red")  # узлы отправитель
        got_net.add_node(dst, dst, title=dst, color="red")  # узлы получатель
    got_net.add_edge(src, dst, value=w)  # ребра отправитель-получатель

neighbor_map = got_net.get_adj_list()

for node in got_net.nodes:
    node_id = node["id"]
    node_neighbors = neighbor_map[node_id]
    neighbor_descriptions = []
    for neighbor in node_neighbors:
        link_data = got_data[(got_data['Payer'] == node_id) & (got_data['Recipient'] == neighbor)]
        sum_tr = link_data['Sum_tr'].values[0]
        num_transactions = link_data['Frequency'].values[0]
        neighbor_description = f"{neighbor} (сумма связи: {sum_tr} руб., количество транзакций: {num_transactions})"
        neighbor_descriptions.append(neighbor_description)
    if neighbor_descriptions:
        node["title"] += f"\nСвязан с:\n" + "\n".join(neighbor_descriptions)
    node["value"] = len(node_neighbors)

for edge in got_net.edges:
    src = edge["from"]
    dst = edge["to"]
    edge_data = got_data[(got_data["Payer"] == src) & (got_data["Recipient"] == dst)]
    num_transactions = edge_data["Frequency"].iloc[0]
    total_amount = edge_data["Sum_tr"].sum()
    edge["title"] = f"Сумма связи: {total_amount} руб.\nКоличество транзакций: {num_transactions}"

df_f = pd.read_csv('prepared_data.csv', delimiter=';')
#got_net.show("Отрисовка графа.html")
got_net.save_graph("Отрисовка графа.html")